# Using the features/factors you take and discovered, e.g., FRED, Fama-French website, ADS, AR, CAPM, momentum factors, volume, price/return lags, etc.) to construct a feature database
- The target variable Y can be either price or return
- Frequency could be either daily or monthly

Team META
* Akhilesh Dongre
* Krishnanand Jha
* Janhvi Shah
* Manav Malavia
* Jaimeen Unagar



## Creating Feature Mart

### Importing Libraries and loading Files

In [ ]:
!pip install fredapi
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=fcf1e94fc701177e8db9a61b86ce6480e17134b89f3b5c70e542cf34de27a266
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import pandas as pd
import numpy as np
from fredapi import Fred
import requests
import ta

In [ ]:
# TRAINING PERIOD: 2022 - 2024
# FEATURE ENGINEERING
# 1. FAMA-FRENCH 5 FACTORS
data_ff5 = pd.read_csv('/content/F-F_Research_Data_5_Factors_2x3_daily.CSV')
data_ff5['date'] = data_ff5['date'].astype(str).str[0:4]+'-'\
+data_ff5['date'].astype(str).str[4:6]+'-'\
+data_ff5['date'].astype(str).str[6:8]
data_ff5['date'] = pd.to_datetime(data_ff5['date'])
# data_ff5['date'] = data_ff5['date'].dt.date
df_ffs = data_ff5.set_index('date')
# 3. ADS INDEX
data_ads = pd.read_excel('/content/ADS_Index_Most_Current_Vintage (1).xlsx')
# manually replace : into -
df_ads = data_ads.set_index('date')
# 4. FRED DATA
fred = Fred(api_key='c6b3423108dc556eb62e679bb130c76f')
# fill with your own FRED API
varList = ['T10Y3M', 'OBMMIJUMBO30YF', # term premium 10yr-3mon, 30 yr mortgage jumbo loan
'DEXUSEU', 'DEXJPUS', 'DEXUSUK', # spot exchange rates to EUR, JPY, GBP
'CBBTCUSD', 'CBETHUSD', # cryptocurrencies
'T10YIE', 'DCOILBRENTEU', # breakeven inflation + brent oil price
'VIXCLS', # implied volatilities
'DAAA', 'DBAA', # corporate bond yield
'NIKKEI225', 'AMERIBOR', 'T10YIE', 'T5YIE',
'BAMLH0A0HYM2','BAMLH0A0HYM2EY',
'DGS10', 'DGS1', 'RIFSPPFAAD90NB', 'DCPN3M',
'DCPF1M', 'DCOILWTICO',
'DHHNGSP', 'USRECD', 'USRECDM', 'USRECDP'] # JPNstock mkt index
SP500 = fred.get_series('SP500')
SP500.name = 'SP500'
df_fred = SP500
data = pd.DataFrame()
# df_fred = SP500# merge data series
for var in varList:
    series = fred.get_series(var)
    series.name = var
    data[var] = series
df_fred = pd.merge(df_fred, data, left_index=True, right_index=True)
start_date = '2022-01-01'
end_date = '2024-03-30'
mask = (df_ffs.index >= start_date) & (df_ffs.index <= end_date)
df_ffs = df_ffs.loc[mask]
DATA_part1 = pd.merge(df_ffs, df_ads, left_index=True, right_index=True)
DATA = pd.merge(DATA_part1, df_fred, left_index=True, right_index=True)
DATA.reset_index(inplace=True)
DATA = DATA.rename(columns = {'index':'Date'})
DATA.to_csv('INFOP7374_FeatureMart.csv', index=True)


In [ ]:
DATA.tail()

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,ADS_Index,SP500,T10Y3M,...,DGS10,DGS1,RIFSPPFAAD90NB,DCPN3M,DCPF1M,DCOILWTICO,DHHNGSP,USRECD,USRECDM,USRECDP
517,2024-01-25,0.46,0.20,0.56,-0.19,0.55,0.022,-0.387875,4894.16,-1.30,...,4.14,4.76,NaN,NaN,NaN,77.91,2.56,0.0,0.0,0.0
518,2024-01-26,-0.02,0.35,-0.27,0.22,-0.02,0.022,-0.377156,4890.97,-1.29,...,4.15,4.78,NaN,NaN,NaN,78.45,2.36,0.0,0.0,0.0
519,2024-01-29,0.85,0.89,-0.59,-1.13,-0.31,0.022,-0.338559,4927.93,-1.34,...,4.08,4.76,5.25,NaN,NaN,77.25,2.41,0.0,0.0,0.0
520,2024-01-30,-0.13,-0.96,0.84,1.00,0.13,0.022,-0.324465,4924.97,-1.36,...,4.06,4.80,NaN,NaN,NaN,78.30,2.26,0.0,0.0,0.0
521,2024-01-31,-1.74,-0.89,-0.30,-0.21,0.37,0.022,-0.309864,4845.65,-1.43,...,3.99,4.73,5.24,NaN,NaN,76.28,2.19,0.0,0.0,0.0


In [ ]:
# META DATA
import yfinance as yf

# Fetch META stock data
meta_data = yf.download('META', start='2022-01-01', end='2024-02-01')
meta_data.reset_index(inplace=True)
meta_data = meta_data.rename(columns = {'index':'Date'})

[*********************100%%**********************]  1 of 1 completed


In [ ]:
meta_data.columns, meta_data.shape

(Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object'),
 (522, 7))

In [ ]:
meta_data["Returns"] = meta_data["Adj Close"] - meta_data["Adj Close"].shift(1)
meta_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns
0,2022-01-03,338.299988,341.079987,337.190002,338.540009,338.181183,14537900,NaN
1,2022-01-04,339.950012,343.089996,331.869995,336.529999,336.173309,15998000,-2.007874
2,2022-01-05,333.019989,335.760010,323.839996,324.170013,323.826416,20564500,-12.346893
3,2022-01-06,322.820007,339.170013,322.720001,332.459991,332.107635,27962800,8.281219
4,2022-01-07,332.739990,337.000000,328.880005,331.790009,331.438354,14722000,-0.669281


Calculating daily returns:

Produce the day's difference of the stock dataframe: (`np.log(meta_data['Open']) - np.log(meta_data['Open'].shift(+1))`)

When we take the logarithm of the ratio between today's closing price and yesterday's, we're essentially computing the daily percentage change in the stock price. Using logarithms in return calculations helps us handle the additive nature of log-returns, making overall return calculations more interpretable and facilitating mathematical operations.

In [ ]:
# Daily return
meta_data["Daily_Return"] = np.log(meta_data["Adj Close"]) - np.log(meta_data["Adj Close"].shift(1))
# meta_data = meta_data.dropna()
meta_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return
0,2022-01-03,338.299988,341.079987,337.190002,338.540009,338.181183,14537900,NaN,NaN
1,2022-01-04,339.950012,343.089996,331.869995,336.529999,336.173309,15998000,-2.007874,-0.005955
2,2022-01-05,333.019989,335.760010,323.839996,324.170013,323.826416,20564500,-12.346893,-0.037419
3,2022-01-06,322.820007,339.170013,322.720001,332.459991,332.107635,27962800,8.281219,0.025252
4,2022-01-07,332.739990,337.000000,328.880005,331.790009,331.438354,14722000,-0.669281,-0.002017


### Feature-set 1: Typical Price, Typical_Price_Return
- `Typical_Price` is the *mean* value of High, Low and Close values

In [ ]:
meta_data["Typical_Price"] = meta_data[["High", "Low", "Close"]].mean(axis=1)
meta_data["Typical_Price_Return"] = (
    np.log(meta_data.Typical_Price) - np.log(meta_data.Typical_Price.shift(+1))
) * 100.0
# meta_data = meta_data.dropna()
meta_data.fillna(meta_data.mean())
meta_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return
0,2022-01-03,338.299988,341.079987,337.190002,338.540009,338.181183,14537900,NaN,NaN,338.936666,NaN
1,2022-01-04,339.950012,343.089996,331.869995,336.529999,336.173309,15998000,-2.007874,-0.005955,337.163330,-0.524579
2,2022-01-05,333.019989,335.760010,323.839996,324.170013,323.826416,20564500,-12.346893,-0.037419,327.923340,-2.778761
3,2022-01-06,322.820007,339.170013,322.720001,332.459991,332.107635,27962800,8.281219,0.025252,331.450002,1.069711
4,2022-01-07,332.739990,337.000000,328.880005,331.790009,331.438354,14722000,-0.669281,-0.002017,332.556671,0.333331


### Feature-set 2: Common Transforms
- `log of volume`
- `pct_change of volume`
- `difference in volume`
- `log of 5 day moving average of volume`
- `Daily volume vs. 200 day moving average`
- `Daily closing price vs. 50 day exponential moving average`

In [ ]:
meta_data["Volume_Log"] = np.log(meta_data.Volume)
meta_data["Volume_Differencing"] = meta_data.Volume.diff()
meta_data["Volume_Differencing_10"] = meta_data.Volume.diff(10)
meta_data["Volumne_Percent_Change"] = meta_data.Volume.pct_change()

In [ ]:
# Log of 5 day moving average of volume
meta_data["MA_5"] = np.log(meta_data.Volume.rolling(5).mean())

# Daily volume vs. 200 day moving average
meta_data["Volumne_MA_200"] = (
    meta_data.Volume / meta_data.Volume.rolling(200).mean() - 1
)

# Daily closing price vs. 50 day Exponential Moving Avg
meta_data["Close_EMA_50"] = meta_data.Close / meta_data.Close.ewm(span=50).mean() - 1

In [ ]:
meta_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,Volume_Log,Volume_Differencing,Volume_Differencing_10,Volumne_Percent_Change,MA_5,Volumne_MA_200,Close_EMA_50
0,2022-01-03,338.299988,341.079987,337.190002,338.540009,338.181183,14537900,NaN,NaN,338.936666,NaN,16.492270,NaN,NaN,NaN,NaN,NaN,0.000000
1,2022-01-04,339.950012,343.089996,331.869995,336.529999,336.173309,15998000,-2.007874,-0.005955,337.163330,-0.524579,16.587974,1460100.0,NaN,0.100434,NaN,NaN,-0.002918
2,2022-01-05,333.019989,335.760010,323.839996,324.170013,323.826416,20564500,-12.346893,-0.037419,327.923340,-2.778761,16.839077,4566500.0,NaN,0.285442,NaN,NaN,-0.026188
3,2022-01-06,322.820007,339.170013,322.720001,332.459991,332.107635,27962800,8.281219,0.025252,331.450002,1.069711,17.146386,7398300.0,NaN,0.359761,NaN,NaN,-0.000944
4,2022-01-07,332.739990,337.000000,328.880005,331.790009,331.438354,14722000,-0.669281,-0.002017,332.556671,0.333331,16.504854,-13240800.0,NaN,-0.473515,16.74708,NaN,-0.002319


### Feature-set 3: Momentum Indicators

**1. AwesomeOscillatorIndicator**


In a real-life trading scenario using MetaStock for Meta Platforms, Inc. (META), the trader would:

1. Set Up AO: Apply the Awesome Oscillator to META's stock chart in MetaStock to analyze momentum.
2. Buy Signal: Enter a long position when the AO crosses above the zero line, indicating increasing bullish momentum.
3. Monitor and Hold: Keep the position as long as the AO stays above zero and watch for saucer formations or twin peaks for ongoing bullish signals.
4. Sell Signal: Exit the position when the AO crosses below zero, suggesting that bullish momentum is waning, thereby securing profits.


In [ ]:
meta_data['Momentum_AwesomeOscillatorIndicator'] = ta.momentum.AwesomeOscillatorIndicator(meta_data.High, meta_data.Low,window1 = 5,window2 = 34, fillna=False).awesome_oscillator()

**2. Kaufman’s Adaptive Moving Average (KAMA)**

Kaufman's Adaptive Moving Average (KAMA) efficiently tracks market prices by adjusting its sensitivity based on observed price volatility, reducing lag in trend detection compared to traditional moving averages. In MetaStock, using KAMA helps traders:

1. Identify Trend Directions: Indicates up or down trends based on the position of the price relative to KAMA.
2. Generate Trading Signals: Entry (buy) and exit (sell) signals are given by price crossovers with KAMA.
3. Filter Out Noise: Adapts to volatility to smooth out noise, providing clearer signals during various market conditions.

In [ ]:
meta_data['Momentum_KAMA'] = ta.momentum.KAMAIndicator(meta_data.Close, fillna=False).kama()

**3. PercentagePriceOscillator**

Percentage Price Oscillator (PPO) is a momentum oscillator that measures the relative difference between two moving averages as a percentage, typically the 26-period and 12-period exponential moving averages (EMAs). This indicator helps highlight convergence, divergence, and crossovers.

Impact on Trading with MetaStock for Meta Platforms, Inc. (META):
1. Trend Identification: PPO crossing above zero suggests an uptrend (bullish), while crossing below indicates a downtrend (bearish).
2. Signal Generation: A positive divergence (when the security is making new lows while the PPO is increasing) can signal a potential reversal to the upside, whereas a negative divergence (new highs with decreasing PPO) suggests a possible downside reversal.
3. Volatility Insight: The width of the PPO histogram (difference between PPO line and its signal line) can indicate the volatility; wider bars suggest more volatile conditions.

In [ ]:
meta_data['Momentum_PercentagePVolumneOscillator'] = ta.momentum.PercentageVolumeOscillator(meta_data.Volume, fillna=False).pvo()

**4. Rate of Change (ROC)**

Rate of Change (ROC) is a momentum oscillator that measures the percentage change in price between the current price and the price a certain number of periods ago. The ROC is used to identify overbought or oversold conditions, trend confirmations, and potential price reversals.

Impact on Trading with MetaStock for Meta Platforms, Inc. (META):
1. Momentum Insights: A rising ROC indicates increasing momentum which suggests a bullish trend, while a falling ROC indicates decreasing momentum, suggesting a bearish trend.
2. Signal for Entry/Exit: ROC crossing above a predefined upper threshold might signal an overbought condition (potential sell), whereas crossing below a lower threshold could indicate an oversold condition (potential buy).
3. Trend Confirmation: Consistently positive ROC values affirm an uptrend, and negative values confirm a downtrend, helping traders decide to hold or exit positions.

In [ ]:
meta_data['Momentum_ROC'] = ta.momentum.ROCIndicator(meta_data.Close, fillna=False).roc()

**5. Relative Strength Index (RSI)**

Relative Strength Index (RSI) is a momentum oscillator that measures the speed and change of price movements on a scale of 0 to 100. Typically, RSI is used to identify overbought or oversold conditions in a stock, with levels marked at 70 (overbought) and 30 (oversold).

Impact on Trading with MetaStock for Meta Platforms, Inc. (META):
1. Overbought/Oversold Identification: An RSI above 70 suggests that META may be overbought and could be set for a price decline, while an RSI below 30 might indicate that META is oversold and potentially due for a price increase.
2. Signal for Reversals: Divergence between RSI and price action (price makes a new high/low, RSI does not) often predicts potential reversals, useful for deciding on entry or exit points.
3. Trend Strength: Consistent RSI levels near high or low extremes can indicate the strength of a trend, helping traders understand the momentum's sustainability.

In [ ]:
meta_data['Momentum_RSI'] = ta.momentum.RSIIndicator(meta_data.Close, fillna=False).rsi()

**6. Stochastic RSI**

Stochastic RSI is an oscillator that measures the level of RSI relative to its high-low range over a set period of time. It provides more sensitivity and identifies more overbought and oversold conditions than traditional RSI.

Impact on Trading with MetaStock for Meta Platforms, Inc. (META):

1. Frequent Signals: StochRSI fluctuates more frequently than the RSI, which can provide earlier signals for overbought (above 0.8) and oversold (below 0.2) conditions.
2. Refined Entries/Exits: Because it signals faster than RSI, it allows traders to enter or exit trades more precisely, capturing better prices on swings.
3. Momentum Cues: Quick crossovers of the StochRSI lines can indicate momentum shifts, useful for short-term trading decisions in volatile stocks like META.

In [ ]:
meta_data['Momentum_StochRSIIndicator'] = ta.momentum.StochRSIIndicator(meta_data.Close, fillna=False).stochrsi()

**7. True strength index (TSI)**

True Strength Index (TSI) is a momentum oscillator that combines direction, magnitude, and velocity of price movements. It is primarily used to identify overbought and oversold conditions through its value and signal line crossovers.

Impact on Trading with MetaStock for Meta Platforms, Inc. (META):

1. Trend Confirmation: Positive TSI values indicate upward momentum (bullish), while negative values suggest downward momentum (bearish).
2. Overbought/Oversold Conditions: Extreme TSI readings (e.g., above +25 or below -25) can signal that META's stock price is in overbought or oversold territory, potentially leading to a price correction.
3. Signal Line Crossovers: Crossovers of the TSI line and its signal line can be used to time entries and exits. A crossover above the signal line can indicate a buy signal, whereas a crossover below the signal line might suggest a sell signal.

In [ ]:
meta_data['Momentum_TSIIndicator'] = ta.momentum.TSIIndicator(meta_data.Close, fillna=False).tsi()

In [ ]:
meta_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,...,Volume_Log,Volume_Differencing,Volume_Differencing_10,Volumne_Percent_Change,MA_5,Volumne_MA_200,Close_EMA_50,Momentum_AwesomeOscillatorIndicator,Momentum_KAMA,Momentum_PercentagePVolumneOscillator
0,2022-01-03,338.299988,341.079987,337.190002,338.540009,338.181183,14537900,NaN,NaN,338.936666,...,16.492270,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
1,2022-01-04,339.950012,343.089996,331.869995,336.529999,336.173309,15998000,-2.007874,-0.005955,337.163330,...,16.587974,1460100.0,NaN,0.100434,NaN,NaN,-0.002918,NaN,NaN,NaN
2,2022-01-05,333.019989,335.760010,323.839996,324.170013,323.826416,20564500,-12.346893,-0.037419,327.923340,...,16.839077,4566500.0,NaN,0.285442,NaN,NaN,-0.026188,NaN,NaN,NaN
3,2022-01-06,322.820007,339.170013,322.720001,332.459991,332.107635,27962800,8.281219,0.025252,331.450002,...,17.146386,7398300.0,NaN,0.359761,NaN,NaN,-0.000944,NaN,NaN,NaN
4,2022-01-07,332.739990,337.000000,328.880005,331.790009,331.438354,14722000,-0.669281,-0.002017,332.556671,...,16.504854,-13240800.0,NaN,-0.473515,16.74708,NaN,-0.002319,NaN,NaN,NaN


### Feature-set 4: Trend Indicators

**1. Average Directional Movement Index (ADX)**

Average Directional Movement Index (ADX)
Average Directional Movement Index (ADX) measures the strength of a trend, regardless of its direction. A value above 25 typically suggests a strong trend, while below 20 indicates a weak trend.

Impact on META Stock:

Trend Strength: Helps traders determine when META is in a strong or weak trend, aiding in strategy adjustments (e.g., trend following in strong trends).
Signal Potency: Enhances the reliability of trend signals from other indicators, signaling more potent trade opportunities when ADX is high.

In [ ]:
meta_data['Trend_ADX'] = ta.trend.ADXIndicator(meta_data.High,meta_data.Low, meta_data.Close, window = 20,fillna=False).adx()

**2. Aroon Indicator**

Aroon Indicator consists of two lines: Aroon Up and Aroon Down. It identifies whether META is in a trend, and its strength, and the likelihood of trend reversal.

Impact on META Stock:

Trend Reversals: Detects potential reversals as Aroon Up crosses above Aroon Down for uptrends, or below for downtrends.
Trend Confirmation: High Aroon Up values indicate a strong uptrend, and high Aroon Down values indicate a strong downtrend.

In [ ]:
meta_data['Trend_AroonIndicator'] = ta.trend.AroonIndicator(meta_data.Close, meta_data.Low, window=20, fillna=False).aroon_indicator()

**3. Commodity Channel Index (CCI)**

Commodity Channel Index (CCI) measures the difference between the current price and the historical average price. High positive values indicate prices are above their average (strong condition), and strong negative values mean the opposite.

Impact on META Stock:

Overbought/Oversold Conditions: CCI above +100 can signal META is overbought, and below -100 indicates oversold, useful for predicting price reversals.
Momentum Indicator: Sudden changes can signal momentum shifts which might indicate the start of a new trend.

In [ ]:
meta_data['Trend_CCI'] = ta.trend.CCIIndicator(meta_data.High, meta_data.Low, meta_data.Close, window = 20,fillna=False).cci()

**4. Detrended Price Oscillator (DPO)**

Detrended Price Oscillator (DPO) isolates short-term fluctuations and cycles by removing the trend effect from the price, helping to identify cycles and overbought/oversold conditions.

Impact on META Stock:

Identify Cycles: Helps spot cycles in META stock prices for timely buys and sells.
Price Extremes: Indicates overbought or oversold conditions, useful for timing the market.

In [ ]:
meta_data['Trend_DPO'] = ta.trend.DPOIndicator(meta_data.Close, window = 20, fillna=False).dpo()

**5. EMA - Exponential Moving Average**

Exponential Moving Average (EMA) focuses more on recent prices, which makes it quicker to react to price changes than a simple moving average.

Impact on META Stock:

Trend Direction: Quick to show changes in trend direction, providing timely signals to traders.
Price Support/Resistance: Acts as dynamic support or resistance levels, helping to time entry and exit in META trades.


In [ ]:
meta_data['Trend_EMA'] = ta.trend.EMAIndicator(meta_data.Close, window = 20, fillna=False).ema_indicator()

**6. Moving Average Convergence Divergence (MACD)**

Moving Average Convergence Divergence (MACD) is used to find trend direction and momentum. It includes two EMAs and a signal line, indicating buy/sell signals through crossover points.

Impact on META Stock:

Trend Following: Effective in identifying buy/sell signals during trending conditions by observing the crossover of MACD and its signal line.
Momentum Shifts: Histogram bars help visualize momentum shifts, aiding in predictive and reactive trading decisions.

In [ ]:
meta_data['Trend_MACD'] = ta.trend.MACD(meta_data.Close, fillna=False).macd()

**7. Mass Index (MI)**

Mass Index (MI) revolves around identifying reversals by comparing range expansions over periods, typically signaling a reversal when it surges above 27.

Impact on META Stock:

Reversal Indicator: Spikes in the MI can forewarn of potential price reversals in META stock, which can be pivotal for high-stakes trading.
Volatility Cues: Helps traders understand periods of upcoming volatility, suggesting more cautious or opportunistic trading strategies.


In [ ]:
meta_data['Trend_MI'] = ta.trend.MassIndex(meta_data.High, meta_data.Low, fillna=False).mass_index()

In [ ]:
meta_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,...,Momentum_RSI,Momentum_StochRSIIndicator,Momentum_TSIIndicator,Trend_ADX,Trend_AroonIndicator,Trend_CCI,Trend_DPO,Trend_EMA,Trend_MACD,Trend_MI
0,2022-01-03,338.299988,341.079987,337.190002,338.540009,338.181183,14537900,NaN,NaN,338.936666,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-04,339.950012,343.089996,331.869995,336.529999,336.173309,15998000,-2.007874,-0.005955,337.163330,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-05,333.019989,335.760010,323.839996,324.170013,323.826416,20564500,-12.346893,-0.037419,327.923340,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-06,322.820007,339.170013,322.720001,332.459991,332.107635,27962800,8.281219,0.025252,331.450002,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-07,332.739990,337.000000,328.880005,331.790009,331.438354,14722000,-0.669281,-0.002017,332.556671,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### Feature-set 5: Volumne Indicator

**1. Chaikin Money Flow (CMF)**

Chaikin Money Flow (CMF) combines price and volume to measure the buying and selling pressure over a set period, usually 20-21 days. A positive CMF indicates buying pressure (bullish sign), while a negative CMF indicates selling pressure (bearish sign).

Impact on META Stock:

Buying/Selling Pressure: A positive CMF value suggests strong buying pressure and potential upward price movement in META stock, whereas a negative value suggests selling pressure and potential decline.
Confirmation Tool: Used alongside other indicators to confirm trend directions or reversals.

In [ ]:
meta_data['Volumne_CMF'] = ta.volume.ChaikinMoneyFlowIndicator(meta_data.High,meta_data.Low,meta_data.Close, meta_data.Volume,window = 20,fillna=False).chaikin_money_flow()

**2. Ease of movement (EoM, EMV)**

Ease of Movement (EoM, EMV) measures the ease of price movement in a stock by comparing volume to price movement; high values indicate little resistance to price change.

Impact on META Stock:

Market Efficiency: High positive EMV values indicate META stock is moving upward efficiently, which might be a buying signal.
Price and Volume Harmony: Highlights periods where price changes align smoothly with volume, often signaling a strong trend.

In [ ]:
meta_data['Volumne_EOM'] = ta.volume.EaseOfMovementIndicator(meta_data.High,meta_data.Low, meta_data.Volume,window = 20,fillna=False).ease_of_movement()

**3. Force Index (FI)**

Force Index (FI) uses price and volume to measure the power behind a price movement. The closer the FI is to zero, the less force behind the movement.

Impact on META Stock:

Price Movement Strength: An increasing FI indicates buyers are strengthening, suggesting a hold or buy strategy; decreasing FI suggests sellers are overpowering.
Trend Confirmation: Crosses above and below zero can confirm potential bullish or bearish trends.

In [ ]:
meta_data['Volumne_FI'] = ta.volume.ForceIndexIndicator(meta_data.Close, meta_data.Volume,window = 20,fillna=False).force_index()

**4. Money Flow Index (MFI)**

Money Flow Index (MFI) is a volume-weighted RSI that combines price and volume to indicate trading pressure and identify overbought or oversold conditions in stock.

Impact on META Stock:

Overbought/Oversold Indicator: MFI values above 80 indicate META is overbought (potential sell signal), while values below 20 suggest oversold (potential buy signal).
Confirm Price Movements: Helps confirm price movements with volume, which is crucial for assessing the strength of a trend.

In [ ]:
meta_data['Volumne_MFI'] = ta.volume.money_flow_index(meta_data.High, meta_data.Low, meta_data.Close,meta_data.Volume, window=20, fillna=False)

**5. Volume Weighted Average Price (VWAP)**

Volume Weighted Average Price (VWAP) gives the average price a stock has traded throughout the day, based on both volume and price. It is often used as a trading benchmark especially by day traders and institutions.

Impact on META Stock:

Trading Benchmark: Indicates whether META is being traded at a premium or discount compared to the average day's price.
Support/Resistance Levels: Often acts as a support or resistance level in intraday trading.
Institutional Attention: Movements away from the VWAP could signal institutional buying or selling pressure.

In [ ]:
meta_data['Volumne_VWAP'] = ta.volume.VolumeWeightedAveragePrice(meta_data.High, meta_data.Low, meta_data.Close,meta_data.Volume, window=20, fillna=False).volume_weighted_average_price()

In [ ]:
meta_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,...,Trend_CCI,Trend_DPO,Trend_EMA,Trend_MACD,Trend_MI,Volumne_CMF,Volumne_EOM,Volumne_FI,Volumne_MFI,Volumne_VWAP
0,2022-01-03,338.299988,341.079987,337.190002,338.540009,338.181183,14537900,NaN,NaN,338.936666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-04,339.950012,343.089996,331.869995,336.529999,336.173309,15998000,-2.007874,-0.005955,337.163330,...,NaN,NaN,NaN,NaN,NaN,NaN,-116.071311,NaN,NaN,NaN
2,2022-01-05,333.019989,335.760010,323.839996,324.170013,323.826416,20564500,-12.346893,-0.037419,327.923340,...,NaN,NaN,NaN,NaN,NaN,NaN,-445.163344,NaN,NaN,NaN
3,2022-01-06,322.820007,339.170013,322.720001,332.459991,332.107635,27962800,8.281219,0.025252,331.450002,...,NaN,NaN,NaN,NaN,NaN,NaN,67.358542,NaN,NaN,NaN
4,2022-01-07,332.739990,337.000000,328.880005,331.790009,331.438354,14722000,-0.669281,-0.002017,332.556671,...,NaN,NaN,NaN,NaN,NaN,NaN,110.034986,NaN,NaN,NaN


### Feature-set 6: Volatility Indicators

**1. Average True Range (ATR)**

Average True Range (ATR) measures market volatility by decomposing the entire range of an asset price for that period. Typically, it captures the essence of price volatility by measuring the range between high and low and considering any gaps in the price action.

Impact on META Stock:

Volatility Indicator: Helps in assessing the volatility of META stock; higher ATR values indicate increased volatility and potentially larger price movements.
Risk Assessment: Useful for setting stop-loss orders based on volatility, ensuring traders are not stopped out prematurely.
Trade Preparation: Provides a gauge for expected price movement which can assist with planning entry and exit points.

In [ ]:
meta_data['Volatility_ATR'] = ta.volatility.AverageTrueRange(meta_data.High, meta_data.Low, meta_data.Close, window=20, fillna=False).average_true_range()

**2. Bollinger Bands**

Bollinger Bands are a set of three lines: the middle band is the SMA of the closing prices, and the upper and lower bands are the standard deviations added and subtracted from the SMA.

Impact on META Stock:

Trend Identification: The price touching or breaking through bands indicates potential continuation or reversal signals.
Volatility Insights: The widening of bands suggests increased volatility, whereas narrowing bands suggest consolidation.
Overbought/Oversold Levels: Prices touching the upper band indicate overbought conditions (possible sell), and the lower band, oversold conditions (possible buy).

In [ ]:
meta_data['Volatility_BB'] = ta.volatility.BollingerBands(meta_data.Close, window=20, fillna=False).bollinger_wband()

**3. Donchian Channel**

Donchian Channel displays the highest high and lowest low over a set period. It is used to understand the momentum and generate breakout signals.

Impact on META Stock:

Breakout Indicator: A price breaking above the upper channel indicates a bullish breakout, suggesting buying META, and below the lower channel suggests a bearish breakout, indicating a selling point.
Support/Resistance Levels: The upper and lower bands form dynamic support and resistance levels, guiding entry and exit strategies.

In [ ]:
meta_data['Volatility_DonchainChannel'] = ta.volatility.DonchianChannel(meta_data.High, meta_data.Low,meta_data.Close, window=20, fillna=False).donchian_channel_wband()

**4. Ulcer Index**

Ulcer Index is a risk measure or downside risk metric that assesses the depth and duration of drawdowns in price. Unlike standard deviation, it only considers downside risk, which is more relevant to investors looking to avoid losses.

Impact on META Stock:

Risk Exposure: Indicates the potential for downside in META stock, helping investors understand the stress or risk of price declines.
Investor Sentiment: High Ulcer Index scores might deter investment during unstable periods, favoring more stable conditions reflected by lower scores.

In [ ]:
meta_data['Volatility_UlcerIndex'] = ta.volatility.UlcerIndex(meta_data.Close, window=20, fillna=False).ulcer_index()

**5. Keltner channel (KC)**

Keltner Channel (KC) uses average true range (ATR) values added to and subtracted from an exponential moving average (EMA) to create channel bands that can serve as dynamic support and resistance levels in the market.

Impact on META Stock:

Trend Indicator: Like Bollinger Bands, crossing the upper or lower KC band can signal a start of a new trend.
Volatility and Trend Strength: Channel width indicates volatility (wider means more volatile) and provides visual cues about trend strength.
Overbought/Oversold Conditions: When META’s price touches or breaches the KC bands, it may indicate overbought or oversold conditions, suggesting potential reversal points.

In [ ]:
meta_data['Volatility_KeltnerChannel'] = ta.volatility.keltner_channel_hband(meta_data.High, meta_data.Low,meta_data.Close, window=20, fillna=False)

In [ ]:
meta_data.shape

(522, 42)

In [ ]:
# meta_data.dropna(inplace = True)
meta_data=meta_data.fillna(meta_data.mean())

In [ ]:
meta_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,...,Trend_AroonIndicator,Trend_CCI,Trend_DPO,Trend_EMA,Volumne_CMF,Volumne_EOM,Volumne_FI,Volatility_ATR,Volatility_BB,Volatility_DonchainChannel
0,2022-01-03,338.299988,341.079987,337.190002,338.540009,338.181183,14537900,0.098935,0.000272,338.936666,...,15.418327,23.678581,-0.31516,222.209561,0.01234,4.278403,-2.344605e+07,0.0,20.717792,20.204587
1,2022-01-04,339.950012,343.089996,331.869995,336.529999,336.173309,15998000,-2.007874,-0.005955,337.163330,...,15.418327,23.678581,-0.31516,222.209561,0.01234,-116.071311,-2.344605e+07,0.0,20.717792,20.204587
2,2022-01-05,333.019989,335.760010,323.839996,324.170013,323.826416,20564500,-12.346893,-0.037419,327.923340,...,15.418327,23.678581,-0.31516,222.209561,0.01234,-445.163344,-2.344605e+07,0.0,20.717792,20.204587
3,2022-01-06,322.820007,339.170013,322.720001,332.459991,332.107635,27962800,8.281219,0.025252,331.450002,...,15.418327,23.678581,-0.31516,222.209561,0.01234,67.358542,-2.344605e+07,0.0,20.717792,20.204587
4,2022-01-07,332.739990,337.000000,328.880005,331.790009,331.438354,14722000,-0.669281,-0.002017,332.556671,...,15.418327,23.678581,-0.31516,222.209561,0.01234,110.034986,-2.344605e+07,0.0,20.717792,20.204587


In [ ]:
meta_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Date                                   522 non-null    datetime64[ns]
 1   Open                                   522 non-null    float64       
 2   High                                   522 non-null    float64       
 3   Low                                    522 non-null    float64       
 4   Close                                  522 non-null    float64       
 5   Adj Close                              522 non-null    float64       
 6   Volume                                 522 non-null    int64         
 7   Returns                                522 non-null    float64       
 8   Daily_Return                           522 non-null    float64       
 9   Typical_Price                          522 non-null    float64   

In [ ]:
DATA.columns

Index(['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'ADS_Index',
       'SP500', 'T10Y3M', 'OBMMIJUMBO30YF', 'DEXUSEU', 'DEXJPUS', 'DEXUSUK',
       'CBBTCUSD', 'CBETHUSD', 'T10YIE', 'DCOILBRENTEU', 'VIXCLS', 'DAAA',
       'DBAA', 'NIKKEI225', 'AMERIBOR', 'T5YIE', 'BAMLH0A0HYM2',
       'BAMLH0A0HYM2EY', 'DGS10', 'DGS1', 'RIFSPPFAAD90NB', 'DCPN3M', 'DCPF1M',
       'DCOILWTICO', 'DHHNGSP', 'USRECD', 'USRECDM', 'USRECDP'],
      dtype='object')

In [ ]:
concatenated_df = pd.concat([DATA, meta_data], axis=1)
concatenated_df=concatenated_df.fillna(concatenated_df.mean())

In [ ]:
concatenated_df

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,ADS_Index,SP500,T10Y3M,...,Trend_AroonIndicator,Trend_CCI,Trend_DPO,Trend_EMA,Volumne_CMF,Volumne_EOM,Volumne_FI,Volatility_ATR,Volatility_BB,Volatility_DonchainChannel
0,2022-01-03,0.73,0.43,0.78,-1.17,0.60,0.000,-0.457452,4796.56,1.55,...,15.418327,23.678581,-0.315160,222.209561,0.012340,4.278403,-2.344605e+07,0.000000,20.717792,20.204587
1,2022-01-04,-0.29,-0.07,3.62,0.63,1.47,0.000,-0.470161,4793.54,1.58,...,15.418327,23.678581,-0.315160,222.209561,0.012340,-116.071311,-2.344605e+07,0.000000,20.717792,20.204587
2,2022-01-05,-2.28,-0.94,2.60,1.41,1.22,0.000,-0.480353,4700.58,1.62,...,15.418327,23.678581,-0.315160,222.209561,0.012340,-445.163344,-2.344605e+07,0.000000,20.717792,20.204587
3,2022-01-06,0.00,0.38,1.75,-0.55,-0.02,0.000,-0.488034,4696.05,1.63,...,15.418327,23.678581,-0.315160,222.209561,0.012340,67.358542,-2.344605e+07,0.000000,20.717792,20.204587
4,2022-01-07,-0.48,-0.96,2.02,-0.12,0.85,0.000,-0.493205,4677.03,1.66,...,15.418327,23.678581,-0.315160,222.209561,0.012340,110.034986,-2.344605e+07,0.000000,20.717792,20.204587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2024-01-25,0.46,0.20,0.56,-0.19,0.55,0.022,-0.387875,4894.16,-1.30,...,80.000000,133.859944,-9.416508,369.700059,0.143397,-91.518306,4.361165e+07,8.290524,15.944191,15.303399
518,2024-01-26,-0.02,0.35,-0.27,0.22,-0.02,0.022,-0.377156,4890.97,-1.29,...,85.000000,129.982282,1.807999,372.027674,0.127338,142.802348,4.066172e+07,8.135998,16.938190,15.401641
519,2024-01-29,0.85,0.89,-0.59,-1.13,-0.31,0.022,-0.338559,4927.93,-1.34,...,90.000000,133.397660,-1.126988,374.788847,0.187458,200.612881,4.906987e+07,8.220698,18.248368,16.968849
520,2024-01-30,-0.13,-0.96,0.84,1.00,0.13,0.022,-0.324465,4924.97,-1.36,...,90.000000,128.885460,1.387988,377.195623,0.158493,180.558218,4.269465e+07,8.149162,18.860406,17.783334


Saving the final dataframe as the *feature mart.*

In [ ]:
filename = f"META_feature_mart.csv"
concatenated_df.to_csv(filename, index=True)

In [ ]:
concatenated_df

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,ADS_Index,SP500,T10Y3M,...,Trend_AroonIndicator,Trend_CCI,Trend_DPO,Trend_EMA,Volumne_CMF,Volumne_EOM,Volumne_FI,Volatility_ATR,Volatility_BB,Volatility_DonchainChannel
0,2022-01-03,0.73,0.43,0.78,-1.17,0.60,0.000,-0.457452,4796.56,1.55,...,15.418327,23.678581,-0.315160,222.209561,0.012340,4.278403,-2.344605e+07,0.000000,20.717792,20.204587
1,2022-01-04,-0.29,-0.07,3.62,0.63,1.47,0.000,-0.470161,4793.54,1.58,...,15.418327,23.678581,-0.315160,222.209561,0.012340,-116.071311,-2.344605e+07,0.000000,20.717792,20.204587
2,2022-01-05,-2.28,-0.94,2.60,1.41,1.22,0.000,-0.480353,4700.58,1.62,...,15.418327,23.678581,-0.315160,222.209561,0.012340,-445.163344,-2.344605e+07,0.000000,20.717792,20.204587
3,2022-01-06,0.00,0.38,1.75,-0.55,-0.02,0.000,-0.488034,4696.05,1.63,...,15.418327,23.678581,-0.315160,222.209561,0.012340,67.358542,-2.344605e+07,0.000000,20.717792,20.204587
4,2022-01-07,-0.48,-0.96,2.02,-0.12,0.85,0.000,-0.493205,4677.03,1.66,...,15.418327,23.678581,-0.315160,222.209561,0.012340,110.034986,-2.344605e+07,0.000000,20.717792,20.204587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2024-01-25,0.46,0.20,0.56,-0.19,0.55,0.022,-0.387875,4894.16,-1.30,...,80.000000,133.859944,-9.416508,369.700059,0.143397,-91.518306,4.361165e+07,8.290524,15.944191,15.303399
518,2024-01-26,-0.02,0.35,-0.27,0.22,-0.02,0.022,-0.377156,4890.97,-1.29,...,85.000000,129.982282,1.807999,372.027674,0.127338,142.802348,4.066172e+07,8.135998,16.938190,15.401641
519,2024-01-29,0.85,0.89,-0.59,-1.13,-0.31,0.022,-0.338559,4927.93,-1.34,...,90.000000,133.397660,-1.126988,374.788847,0.187458,200.612881,4.906987e+07,8.220698,18.248368,16.968849
520,2024-01-30,-0.13,-0.96,0.84,1.00,0.13,0.022,-0.324465,4924.97,-1.36,...,90.000000,128.885460,1.387988,377.195623,0.158493,180.558218,4.269465e+07,8.149162,18.860406,17.783334
